In [1]:
#JSONとOAuth認証用のライブラリを使用
import json
from requests_oauthlib import OAuth1Session
# import setting
import tweepy
import os

# twitter = OAuth1Session(setting.CONSUMER_KEY, setting.CONSUMER_SECRET, setting.ACCESS_TOKEN, setting.ACCESS_SECRET)

In [2]:
import torch
from torch import nn
from transformers import BertTokenizer, BertModel, BertForMaskedLM, tokenization_bert
import numpy as np
from torchtext import data
from torchtext import datasets
from torchtext.vocab import GloVe
import random

model = BertModel.from_pretrained('bert/Japanese_L-12_H-768_A-12_E-30_BPE_transformers')
tokenizer_bert  = BertTokenizer("bert/Japanese_L-12_H-768_A-12_E-30_BPE_transformers/vocab.txt",
                               do_lower_case=False, do_basic_tokenize=False)

In [3]:
import re
import string
import torchtext
# from bert import BertTokenizer
# from bert import BertTokenizer

def preprocessing_text(text):
    text = re.sub("<br/>", "", text)
    
    for p in string.punctuation:
        if (p==".")or (p==","):
            continue
        else:
            text=text.replace(p, "")

    return text

def tokenizer_with_preprocessing(text, tokenizer=tokenizer_bert.tokenize):
    text = preprocessing_text(text)
    ret = tokenizer(text)
    return ret

max_length = 256



In [4]:
TEXT = data.Field(sequential=True, tokenize=tokenizer_with_preprocessing, lower=True)
LABEL = data.Field(sequential=False, use_vocab=False)

In [37]:
!head -c 2500 ../tweet_extructor/twitterJSA_data.json # 文字化けは気にしなくて良いはず

[{"id": 10025, "topic": 10000, "status": 522407718091366400, "label": [0, 0, 1, 1, 0], "text": "\u30a8\u30af\u30b9\u30da\u30ea\u30a2\u306eGPS\u5357\u5317\u304c\u9006\u306b\u306a\u308b\u306e\u306f\u30c7\u30d5\u30a9\u3060\u3063\u305f\u306e\u304b\u3002"}, {"id": 10026, "topic": 10000, "status": 522407768003592192, "label": [0, 0, 1, 0, 0], "text": "xperia\u3067\u30b9\u30af\u30d5\u30a7\u30b9\u7cde\n\u53cd\u5fdc\u9045\u3044\u3093\u3060\u3088\u7cde\u304c"}, {"id": 10027, "topic": 10000, "status": 522408018642628609, "label": [0, 0, 1, 1, 0], "text": "\u590f\u6625\u90fd\u304c\u6301\u3063\u3066\u305f\u30a8\u30af\u30b9\u30da\u30ea\u30a2\u3082\u4eca\u4f7f\u3046\u306b\u306f\u8f9b\u3044"}, {"id": 10028, "topic": 10000, "status": 522408394871672832, "label": [0, 0, 0, 1, 0], "text": ""}, {"id": 10029, "topic": 10000, "status": 522408454778929153, "label": [0, 0, 0, 1, 0], "text": ""}, {"id": 10030, "topic": 10000, "status": 522408539814260736, "label": [0, 1, 0, 0, 0], "text": ""}, {"id": 10031, "t

In [38]:
# 今回は一つのjsonデータを読み込む
dataset = torchtext.data.TabularDataset.splits(
    path='../tweet_extructor/',
    train="twitterJSA_data.json",
    format='JSON', fields= {"text": ("text", TEXT), "label":("label", LABEL)} )



# なぜかsplitで返り値が一つしかかえらない(??)

In [39]:
# train_ds, val_ds = dataset[0].split() # ので、この式を実行するとエラー仕方ないので、下を実行
# train_ds= dataset[0].split()

In [40]:
!ls bert

Japanese_L-12_H-768_A-12_E-30_BPE_transformers


In [41]:
len(tokenization_bert.load_vocab(vocab_file="./bert/Japanese_L-12_H-768_A-12_E-30_BPE_transformers/vocab.txt"))

32006

In [42]:
# BERT用の単語辞書を辞書型変数に用意します
# from bert import  load_vocab
# vocab_bert, ids_to_tokens_bert = tokenization_bert.load_vocab(vocab_file="./bert/Japanese_L-12_H-768_A-12_E-30_BPE_transformers/vocab.txt")
vocab_bert = tokenization_bert.load_vocab(vocab_file="./bert/Japanese_L-12_H-768_A-12_E-30_BPE_transformers/vocab.txt")

# 1度適当にbuild_vocabでボキャブラリーを作成してから、BERTのボキャブラリーを上書きします
TEXT.build_vocab("", min_freq=1)
TEXT.vocab.stoi = vocab_bert

In [43]:
train_dl = torchtext.data.Iterator(
#     train_ds[0], batch_size=3, train=True)
    dataset[0], batch_size=3, train=True)

# ここで結局エラー・・・

In [44]:
# 動作確認 検証データのデータセットで確認
batch = next(iter(train_dl))
print(batch.Text)
print(batch.Label)

KeyError: 'エクスペリアのgps南北が逆になるのはデフォだったのか。'

In [ ]:
for i in train_dl:
    print(i)

In [ ]:
class BertForTwitterSuzuki(nn.Module):
    '''BERTモデルにTwitterのポジ・ネガ系５分類を判定する部分をつなげたモデル'''

    def __init__(self, net_bert):
        super(BertForIMDb, self).__init__()

        # BERTモジュール
        self.bert = net_bert  # BERTモデル

        # headにポジネガ予測を追加
        # 入力はBERTの出力特徴量の次元、出力はポジ・ネガの2つ
        self.cls = nn.Linear(in_features=768, out_features=5)

        # 重み初期化処理
        nn.init.normal_(self.cls.weight, std=0.02)
        nn.init.normal_(self.cls.bias, 0)

    def forward(self, input_ids, token_type_ids=None, attention_mask=None, output_all_encoded_layers=False, attention_show_flg=False):
        '''
        input_ids： [batch_size, sequence_length]の文章の単語IDの羅列
        token_type_ids： [batch_size, sequence_length]の、各単語が1文目なのか、2文目なのかを示すid
        attention_mask：Transformerのマスクと同じ働きのマスキングです
        output_all_encoded_layers：最終出力に12段のTransformerの全部をリストで返すか、最後だけかを指定
        attention_show_flg：Self-Attentionの重みを返すかのフラグ
        '''

        # BERTの基本モデル部分の順伝搬
        # 順伝搬させる
        if attention_show_flg == True:
            '''attention_showのときは、attention_probsもリターンする'''
            encoded_layers, pooled_output, attention_probs = self.bert(
                input_ids, token_type_ids, attention_mask, output_all_encoded_layers, attention_show_flg)
        elif attention_show_flg == False:
            encoded_layers, pooled_output = self.bert(
                input_ids, token_type_ids, attention_mask, output_all_encoded_layers, attention_show_flg)

        # 入力文章の1単語目[CLS]の特徴量を使用して、ポジ・ネガを分類します
        vec_0 = encoded_layers[:, 0, :]
        vec_0 = vec_0.view(-1, 768)  # sizeを[batch_size, hidden_sizeに変換
        out = self.cls(vec_0)

        # attention_showのときは、attention_probs（1番最後の）もリターンする
        if attention_show_flg == True:
            return out, attention_probs
        elif attention_show_flg == False:
            return out